# NEMO

* https://www.nemo-ocean.eu/
* [forge](http://forge.ipsl.jussieu.fr/nemo/wiki/Users)

In [5]:
NEMO="svn co http://forge.ipsl.jussieu.fr/nemo/svn/NEMO/releases/release-3.6/NEMOGCM"
XIOS="svn co -r819 http://forge.ipsl.jussieu.fr/ioserver/svn/XIOS/trunk xios-2.0"
XIOS_MAKE="cd xios-2.0 && ./make_xios --job 8 --arch XC40_Cray"

In [1]:
%%bash
XIOS="svn co -r819 http://forge.ipsl.jussieu.fr/ioserver/svn/XIOS/trunk xios-2.0"
[ ! -d xios-2.0 ] && $XIOS
cd xios-2.0 
# ./make_xios --avail
# arch=GCC_LINUX ./make_xios 
# ./make_xios --job 8 --arch XC40_Cray


In [7]:
######################
# from __future__ import print_function
import os, sys;
lib = "/usr/local/apps/ecflow/current/lib/python2.7/site-packages/ecflow"
lib = "/usr/local/apps/ecflow/current/lib/python3.5/site-packages/ecflow"
lib = "/usr/local/lib/python3.5/site-packages/ecflow"
sys.path.append(lib)
import ecf; from ecf import (Client, Defs, Suite, Family, Task, Defstatus, Edit, Trigger)
try: x = Edit(test="value")  # Edit is present in recent ecf.py module
except:
    class Edit(Variables): pass
home = os.getenv("HOME") + "/ecflow_server"
user = os.getenv("USER")

# USERNAME and PASS variables are stored in a rc file under HOME directory
# vim ~/.nemorc; chmod 700 ~/.nemorc # hide access to password
CMD = "svn co http://forge.ipsl.jussieu.fr/nemo/svn/NEMO/releases/release-3.6/NEMOGCM"
# CMD is here a PYTHON VARIABLE
MAKE = "   cd NEMOGCM/CONFIG; ./makenemo –m 'my_arch' –r GYRE -n 'MY_GYRE'"
# SUITE
node = Suite("NEMO").add(
    Defstatus("suspended"),
    Edit(ECF_HOME=home, ECF_INCLUDE=home + "/include", ECF_FILES=home + "/files",
         ECF_EXTN=".ecg",  # current convention for generated task template extension
         ECF_JOB_CMD="%ECF_JOB% > %ECF_JOBOUT% 2>&1",  # localhost run
         ECF_URL_CMD="firefox %URL%",  # web in ecflow GUI can call the web-browser on the right page
         URL="https://www.nemo-ocean.eu/", ),
    Family("make").add(
        Family("get").add(Task("cmd").add(
            Edit(CMD=CMD,  # PYTHON VARIABLE IS TURNED INTO ECFLOW VARIABLE
                 ARGS="")), ),
        Family("compile").add(
            Trigger(["get"]),
            Task("cmd").add(
                Edit(CMD=MAKE)), #"cd script_install && chmod +x install*sh && ./install_lmdz.sh")), 
        ), ),
    Family("main").add(Task("cmd").add(Edit(CMD="echo", ARGS="YOUR PART"))))

# TASK TEMPLATE
with open(home + "/files/cmd.ecg", 'w') as task_template: print("""#!/bin/bash
%include <head.h>
 %CMD:echo% %ARGS:%  # THE TASK TEMPLATE USES THE SUITE VARIABLE 
%include <tail.h>""", file=task_template)

# DEFS
defs = Defs()
defs.add_suite(node)
path = '/' + node.name()

# CLIENT
client = Client("localhost@%s" % os.getenv("ECF_PORT", 2500))  # PYTHON CLIENT
client.replace(path, defs)  # load/replace the top node (suite)
client.begin_suite(node.name())  # BEGIN suite: UNKNOWN -> QUEUED
client.resume(path) # RESUME suite: SUSPENDED -> create job and submit

In [3]:
os.system("ecflow_ui &")

0